In [1]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler
from pip._internal import main as pipmain

In [2]:
def data_extract(path_to_db):
    conn = sqlite3.connect(path_to_db)

    query = """
        SELECT
        e."Customer Identity",
        "First Policy´s Year",
        "Brithday Year",
        "Educational Degree",
        "Gross Monthly Salary",
        "Geographic Living Area",
        "Has Children (Y=1)",
        "Customer Monetary Value",
        "Claims Rate",
        l."Premiums in LOB: Motor",
        l."Premiums in LOB: Household",
        l."Premiums in LOB: Health",
        l."Premiums in LOB:  Life",
        l."Premiums in LOB: Work Compensations"
        FROM
        Engage AS e
        JOIN LOB AS l ON l."Customer Identity" = e."Customer Identity"
        ORDER BY
        e."Customer Identity";
    """

    data_df = pd.read_sql_query(query, conn)
    df = data_df.copy()  # let's keep a copy of the original data

    # remaining column names to manageable variable names
    column_names = ['ID', 'First_Policy', 'Birthday', 'Education',
                    'Salary', 'Area', 'Children', 'CMV', 'Claims',
                    'Motor', 'Household', 'Health', 'Life',
                    'Work_Compensation']
    # renaming the columns
    df.columns = column_names
    # seting 'ID' as index
    df.set_index('ID', inplace=True, drop=True)
    return data_df, df

In [3]:
#preprocessing funcs

def cleaning_df(df):
    # turning impossible values into NaN
    df.loc[df["Birthday"] < 1900, "Birthday"] = np.nan
    df.loc[df["First_Policy"] > 2020, "First_Policy"] = np.nan
    # turning Education into numeric
    df["Education"] = df["Education"].str.extract(r"(\d)").astype(np.float)
    return df


def add_dummies(df, cols):
    """Adds dummy columns to selected variables using the One Hot Encoding method.
    Drops the first column."""
    df_with_dummies = pd.get_dummies(df, columns=cols, drop_first=True)
    return df_with_dummies


def outlier_conditions(df):
    """
    Sets the condition for the identification of outliers in a dataframe
    """
    return ~(np.abs(df - df.mean()) > (3 * df.std()))


def remove_outliers(df, cols):
    """
    Replaces outliers by NaNs.
    Selected columns must be numerical.
    """
    outlier_df_cond = outlier_conditions(df)
    outliers_count = (
        (df[cols] == df[outlier_df_cond][cols]) == False
        )[cols].sum()
    
    temp_df = df[cols].copy()
    outlier_tempdf_cond = outlier_conditions(temp_df)
    temp_df = temp_df[outlier_tempdf_cond]
    
    df.loc[:, cols] = temp_df.loc[:, cols].copy()
    return df, outliers_count


def handle_nans(df, cols):
    """
    Replaces NaNs by column mean.
    Selected columns must be numerical.
    """
    df.fillna(df.mean()[cols], inplace=True)
    return df


def standardize_data(df, cols):
    """Standardizes data from `cols`.
    cols -> list
    """
    df[cols] = StandardScaler().fit_transform(df[cols])
    return df


def preprocessing_df(df):
    df = cleaning_df(df)
    df, outliers_count = remove_outliers(df, ['Motor', 'Household', 'Health', 'Life', 'Work_Compensation'])
    df = handle_nans(df, ["Salary", "First_Policy", "Birthday", "Children", 'Motor', 'Household', 'Health', 'Life', 'Work_Compensation'])
    df = standardize_data(df, ['Motor', 'Household', 'Health', 'Life', 'Work_Compensation'])
    df = add_dummies(df, ['Area', 'Education'])
    # duplicated rows (showing only the duplicates)
    # dups_df = df[df.duplicated(keep="first")].copy()
    return df, outliers_count

In [4]:
pd.set_option('display.width', 4000)
pd.set_option('max_colwidth', 4000)
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 200)
pd.set_option('display.float_format', '{:.2f}'.format)

my_path = r'.\data\insurance.db'
profile = False

_, df = data_extract(my_path)

if profile:
    try:
        import pandas_profiling
    except ImportError as e:
        print(e.args)
        pipmain(['install', 'pandas_profiling'])
        import pandas_profiling
    prof = df.profile_report(style={'full_width': True}, title='Pandas Profiling Report')
    prof.to_file(output_file="./out/df_profiling.html")

df, outliers_count = preprocessing_df(df)

In [5]:
outliers_count

Motor                 40
Household             36
Health                46
Life                 314
Work_Compensation    245
dtype: int64

In [6]:
df.head()

,First_Policy,Birthday,Salary,Children,CMV,Claims,Motor,Household,Health,Life,Work_Compensation,Area_2.0,Area_3.0,Area_4.0,Education_2.0,Education_3.0,Education_4.0
ID,,,,,,,,,,,,,,,,,
1,1985.00,1982.00,2177.00,1.00,380.97,0.39,0.57,-0.55,-0.29,0.24,-0.54,0,0,0,1,0,0
2,1981.00,1995.00,677.00,1.00,-131.13,1.12,-1.59,0.96,-0.69,0.00,1.75,0,0,1,1,0,0
3,1991.00,1970.00,2277.00,0.00,504.67,0.28,-0.66,0.10,-0.58,1.27,1.57,0,1,0,0,0,0
4,1990.00,1981.00,1099.00,1.00,-16.99,0.99,-0.83,-0.71,1.92,-0.07,-0.24,0,0,1,0,1,0
5,1986.00,1973.00,1763.00,1.00,35.23,0.90,0.30,-0.69,0.20,-0.50,0.09,0,0,1,0,1,0


In [7]:
df.columns

Index(['First_Policy', 'Birthday', 'Salary', 'Children', 'CMV', 'Claims', 'Motor', 'Household', 'Health', 'Life', 'Work_Compensation', 'Area_2.0', 'Area_3.0', 'Area_4.0', 'Education_2.0', 'Education_3.0', 'Education_4.0'], dtype='object')

In [14]:
df.isna().any().any()

False